In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64
import pandas as pd
import animal_shelter

# Import your CRUD Python module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 33911

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port)

# Read all records from the database
df = pd.DataFrame.from_records(db.read({}))

# Drop MongoDB '_id' column to avoid Dash DataTable crashes
df.drop(columns=['_id'], inplace=True, errors='ignore')

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load and encode Grazioso Salvare logo image
image_filename = 'Grazioso Salvare Logo.png'  # Replace with your actual image file
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Layout structure
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard - Brian Voo'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '150px'})),
    html.Hr(),
    
    # Interactive filter options (Radio Buttons)
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset',
        labelStyle={'display': 'inline-block', 'margin': '10px'}
    ),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        sort_action='native',
        filter_action='native'
    ),
    html.Br(),
    html.Hr(),

    # Side-by-side chart and map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Queries for each rescue type
water_rescue_query = {
    "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
    "sex_upon_outcome": "Intact Male",
    "age_upon_outcome_in_weeks": {"$lte": 156}
}
mountain_rescue_query = {
    "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
    "sex_upon_outcome": "Intact Male",
    "age_upon_outcome_in_weeks": {"$lte": 156}
}
disaster_tracking_query = {
    "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
    "sex_upon_outcome": "Intact Male",
    "age_upon_outcome_in_weeks": {"$lte": 156}
}

# Callback to update data table based on filter selection
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'water':
        results = db.read(water_rescue_query)
    elif filter_type == 'mountain':
        results = db.read(mountain_rescue_query)
    elif filter_type == 'disaster':
        results = db.read(disaster_tracking_query)
    else:
        results = db.read({})
    df_filtered = pd.DataFrame.from_records(results).drop(columns=['_id'], errors='ignore')
    return df_filtered.to_dict('records')

# Callback to create chart based on data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return []
    df_view = pd.DataFrame(viewData)
    if df_view.empty or 'breed' not in df_view.columns:
        return []
    fig = px.pie(df_view, names='breed', title='Rescue Dog Breed Distribution')
    return [dcc.Graph(figure=fig)]

# Highlight column in data table when selected
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return[{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback to update map based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return []
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row]['name'])
                ])
            ])
        ])
    ]

# Run server
app.run_server(debug=True)


Dash app running on http://127.0.0.1:15749/
